In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
f = open("/kaggle/input/glove840b300dtxt/glove.840B.300d.txt", "r")
lines = f.readlines()
i = 0;
for line in lines:
    i = i + 1
    if(i > 10):
        break
    print(line)

In [ ]:
import pickle

file = open("/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl", 'rb')

data = pickle.load(file)

file.close()
i = 0;
print(type(data))
for key in data:
    i = i + 1
    if(i > 10):
        break
    print(key, type(data[key]), data[key])

In [ ]:
import tensorflow as tf
print(tf.add(1,2))
print(tf.square(5).dtype)
print(tf.reduce_sum([1, 2, 3]))
print(tf.square(2) + tf.square(3))

In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(type(x))

In [ ]:
import numpy as np
ndarray = np.ones([3,3])
tensor = tf.multiply(ndarray, 42)
print(tensor)
print(np.add(tensor, 1))
print(tensor.numpy())

In [ ]:
x = tf.random.uniform([3, 3])
print(tf.config.experimental.list_physical_devices("GPU"))
print(x.device.endswith('GPU:0'))

In [ ]:
import time
import tensorflow as tf

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    result = time.time() - start
    print("10 loops: {:0.2f}ms".format(1000*result))

print("On CPU")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
    Line2
    Line3
    """)

ds_file = tf.data.TextLineDataset(filename)

ds_tensors = ds_tensors.shuffle(3).batch(3)

ds_file = ds_file.batch(2)

print('Elements of ds_tensor:')
for x in ds_tensors:
    print(x)

print('\nElements in ds_file')
for x in ds_file:
    print(x)
    

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [ ]:
x = tf.ones((2,2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dx = t.gradient(z, x)
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

Custom Training

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [ ]:
x = tf.zeros([10,10])
x += 2
print(x)

In [ ]:
v = tf.Variable(1.0)
print(v)
v.assign(3.0)
print(v)
v.assign(tf.square(v))
assert v.numpy() == 9.0
print(v)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

class Model(object):
    def __init__(self):
        self.W = tf.Variable(5.0)
        self.b = tf.Variable(0.0)
    def __call__(self, x):
        return self.W * x + self.b

model = Model()

assert model(3.0).numpy() == 15.0

def loss(predict_y, target_y):
    return tf.reduce_mean(tf.square(predict_y - target_y))

TRUE_W = 3.0
TRUE_B = 2.0
NUM_EXAMPLES = 1000

inputs = tf.random.normal(shape = [NUM_EXAMPLES])
noise   = tf.random.normal(shape=[NUM_EXAMPLES])
outputs = inputs * TRUE_W + TRUE_B + noise

import matplotlib.pyplot as plt

plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: %1.6f' % loss(model(inputs), outputs).numpy())



def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
    grads = t.gradient(current_loss, tf.compat.v1.trainable_variables(scope=None))
    optimizer = tf.optimizers.Adam(learning_rate)
    optimizer.apply_gradients(zip(grads,tf.compat.v1.trainable_variables(scope=None)))
    

Ws, bs = [], []
epochs = range(10)
for epoch in epochs:
    Ws.append(model.W.numpy())
    bs.append(model.b.numpy())
    current_loss = loss(model(inputs), outputs)
    
    train(model, inputs, outputs, learning_rate = 0.1)
    print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
        (epoch, Ws[-1], bs[-1], current_loss))

plt.plot(epochs, Ws, 'r',
         epochs, bs, 'b')
plt.plot([TRUE_W] * len(epochs), 'r--',
         [TRUE_B] * len(epochs), 'b--')
plt.legend(['W', 'b', 'True W', 'True b'])
plt.show()

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras

import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras import layers


TRUE_W = 3.0
TRUE_B = 2.0
NUM_EXAMPLES = 1000
"""
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
    def __call__(self, x):
        return self.W * x + self.b
"""
inputs_k  = keras.Input(shape = (NUM_EXAMPLES,))
outputs_k = layers.Dense(1)(inputs_k)
model = keras.Model(inputs=inputs_k, outputs=outputs_k)

def loss(predict_y, target_y):
    return tf.reduce_mean(tf.square(predict_y - target_y))


inputs = tf.random.normal(shape = [1,NUM_EXAMPLES])
noise   = tf.random.normal(shape=[1,NUM_EXAMPLES])
outputs = inputs * TRUE_W + TRUE_B + noise

print('Current loss: %1.6f' % loss(model(inputs), outputs).numpy())
#print(model.trainable_variables)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
    grads  = t.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

epochs = range(10)
for epoch in epochs:
    current_loss = loss(model(inputs), outputs)
    train(model, inputs, outputs, learning_rate = 0.0001)
    print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
        (epoch, Ws[-1], bs[-1], current_loss))


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.keras.backend.clear_session()  # For easy reset of notebook state.

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
#print(model.trainable_variables)

a = tf.random.normal(shape = [1,784])
print(a.shape)
print(model.predict(a))